## Imports

In [1]:
import sys
import os, shutil
import pandas as pd
import time
import math
import random
import itertools
# from gurobipy import *
from math import sqrt
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# import xlrd
# from gurobipy import *
from math import sqrt
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# import xlrd
import matplotlib.pyplot as plt
import numpy as np
# import scipy.stats as stats
import math
import csv

data_path = f'{os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))}/data'
data_path
scenario = 'results_cleaned_20210612'

## Main Function

Set paths and change variables here, then execute the next cell

In [2]:



# #Set parameters
# N = 2000
# As = N/660

# s1 = 4
# s2 = 4
# s3 = 30
# mu1 =  1/s1
# mu2 = 1/s2
# mu3 = 1/s3
# k1 = 12
# k2 = 16
# k3 = 100


# print(math.factorial(4))
# print(pow(4,3))

In [3]:
def simulateQApprox(N,s1,s2,s3,k1,k2,k3, ptile='Avg'):
    As = N/840
    mu1 =  1/s1
    mu2 = 1/s2
    mu3 = 1/s3
    rho1 = As/(k1*mu1)
    rho2 = As/(k2*mu2)
    rho3 = As/(k3*mu3)
    T = 30
    
    if rho1 >= 1:
        return (N,s1,s2,s3,k1,k2,k3,'nan','nan','nan','nan')
    if rho2 >= 1:
        return (N,s1,s2,s3,k1,k2,k3,'nan','nan','nan','nan')
    if rho3 >= 1:
        return (N,s1,s2,s3,k1,k2,k3,'nan','nan','nan','nan')
    
    #Compute W1, 
    One_over_P0 = pow(k1*rho1,k1)/((1-rho1)*(math.sqrt(2*math.pi*k1)*(k1/math.e)**k1))
    for m in range(1,k1):
        One_over_P0 += pow(k1*rho1,m)/(math.sqrt(2*math.pi*m)*(m/math.e)**m)
    P0 = 1/One_over_P0
    L1 = P0*rho1*pow(k1*rho1,k1)/((math.sqrt(2*math.pi*k1)*(k1/math.e)**k1)*pow(1-rho1,2))
    W1 = L1 / As
    
    #Compute W2, 
    One_over_P0 = pow(k2*rho2,k2)/((1-rho2)*(math.sqrt(2*math.pi*k2)*(k2/math.e)**k2))
    for m in range(1,k2):
        One_over_P0 += pow(k2*rho2,m)/(math.sqrt(2*math.pi*m)*(m/math.e)**m)
    P0 = 1/One_over_P0
    L2 = P0*rho2*pow(k2*rho2,k2)/(((math.sqrt(2*math.pi*k2)*(k2/math.e)**k2))*pow(1-rho2,2))
    W2 = L2 / As
    
    #Compute W3,
    try:
        P3_num = k3*pow(As,k3) / (pow(mu3,k3-1)*(k3*mu3-As)*((math.sqrt(2*math.pi*k3)*(k3/math.e)**k3)))

        P3_denom = P3_num
        for m in range(1,k3):
            P3_denom += pow(As,m)/(pow(mu3,m)*(math.sqrt(2*math.pi*m)*(m/math.e)**m))

        P3 = P3_num / P3_denom
        W3 = (s3/k3+1)*P3/(1-rho3)
        L3 = W3*As

    except:
        if (k3 - As*T) >= 0:
            W3 = 0
            L3 = -1
        else:
            W3 = -1
            L3 = 0
    
    
    # If this is a constant multiplier, can we have a table for different percentiles and compute on the fly?
    W190 = W1*(2.645)
    W195 = W1*(2.960)
    W290 = W2*(2.645)
    W295 = W2*(2.960)
    W390 = W3*(2.645)
    W395 = W3*(2.960)
    L390 = L3*(2.645)
    L395 = L3*(2.960)
    
    W1s = math.ceil(W1*60) 
    W2s = math.ceil(W2*60) 
    W3s = math.ceil(W3*60) 
    W190s = math.ceil(W190*60) 
    W290s = math.ceil(W290*60) 
    W390s = math.ceil(W390*60) 
    W195s = math.ceil(W195*60) 
    W295s = math.ceil(W295*60) 
    W395s = math.ceil(W395*60) 
    L390 =  math.ceil(L390)
    L395 =  math.ceil(L395)
    
    if ptile == 'Avg':
        return (N,s1,s2,s3,k1,k2,k3,W1s,W2s,W3s,L3)
    elif ptile == 90:
        return (N,s1,s2,s3,k1,k2,k3,W190s,W290s,W390s,L390)
    elif ptile == 95:
        return (N,s1,s2,s3,k1,k2,k3,W195s,W295s,W395s,L395)
    
    
# print(simulateQ(4000,3,3,15,20,20,172))


In [4]:
fields = ['NoPerDay', 'RegistrationTime', 'VaccineTime', 'WaitingTime' ,'RegistrationDesks','VaccineDesks','SeatingCap','W1','W2','W3','QueueOutside','measure'] 
shutil.rmtree(f'{data_path}/{scenario}')
os.mkdir(f'{data_path}/{scenario}')
    
for N0 in range(1500, 2501, 50): # NoPerDay
    for ptile in ['Avg']: #['Avg', 90, 95]: # percentile response
        with open(f'{data_path}/{scenario}/NoPerDay_{N0}_measure_{ptile}.csv', 'w+') as f:
            csv_writer = csv.writer(f)
            csv_writer.writerow(fields)

            for k10 in range (7, 21, 1): # RegistrationDesks
                for k20 in range(8, 21, 1): # VaccineDesks
                    for k30 in range(40, 251, 2): # SeatingCap
                        for s1 in range(6, 11): #range(12, 21): # range(6, 11): # RegistrationTime
                            s1_adj = s1/2 # s1/4 # s1/2 
                            for s2 in range(6, 11): #range(12, 21): # range(6, 11): # VaccineTime
                                s2_adj = s2/2 # s2/4 # s2/2
                                for s3 in range(15, 31, 5): # WaitingTime Observation area
                                    try:
                                        x=simulateQApprox(N0,s1_adj,s2_adj,s3,k10,k20,k30, ptile)
                                        if x[-1] != 'nan':
                                            row = [f'{val:0.2f}' if type(val)==float else f'{val}' for val in x]
                                            row.extend([ptile])
                                            csv_writer.writerow(row)
                                    except Exception as e:
                                        print(e)
                                        print(N0,s1_adj,s2_adj,s3,k10,k20,k30, ptile)
                                        raise e

    print(f"N0: {N0}")
    break




N0: 1500


In [ ]:
# k3 = 172
# As = 2.272727272727273 
# mu3 = 0.06666666666666667

# print(pow(As,k3))
# print(pow(mu3,k3-1))
# print(k3*mu3-As)
# print(math.sqrt(2*math.pi*k3))
# print(k3/math.e)
# print((math.sqrt(2*math.pi*k3)*(k3/math.e)**k3))

# k3*pow(As,k3) / (pow(mu3,k3-1)*(k3*mu3-As)*((math.sqrt(2*math.pi*k3)*(k3/math.e)**k3)))

